# Final Exam/Project -- Applied Machine Learning (CNN Portion)

Josh Gregory

## Overview

For the specific convolutional neural network (CNN) architecture, I'm going to **DESCRIPTION GOES HERE**.

In [5]:
import numpy as np
import pandas as pd
import os
import pickle
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# Doing experiment tracking in Weights and Biases, which is the same that I'm using for my thesis. Trying to use this to get some experience using it
import wandb
from wandb.integration.keras import WandbMetricsLogger

# Also going to hyperparameter optimize using Optuna, again this is the same library that I'm using as in my thesis, so I'm going to use it here to get familiar with it.
import optuna

/home/josh/anaconda3/envs/applied_ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
wandb.login()

wandb: Currently logged in as: jag42 (jag42-university-of-colorado-boulder). Use `wandb login --relogin` to force relogin


True

I'm running this notebook on my laptop, which has a discrete NVIDIA GPU. Let's make sure TensorFlow can see it:

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Pre-process images:

In [2]:
def pre_process_images(data_directory):
    features = []
    labels = []

    for color in ['red', 'yellow', 'green']:
        color_path = os.path.join(data_directory, color)

        # Get all files in color directory

        for file in os.listdir(color_path):
            img_path = os.path.join(color_path, file)
                
            # Read in image
            img = cv2.imread(img_path)

            image_crop = np.copy(img)
            row_crop = 7
            col_crop = 8
            image_crop = img[row_crop:-row_crop, col_crop:-col_crop, :]

            img_resized = cv2.resize(image_crop, (32, 32))

            # Resize image
            # img_resized = transform.resize(img, (32, 32))

            # Flatten image
            flat_features = img_resized.flatten()
            features.append(flat_features)

            # Append the label as well
            labels.append(color)

    features = np.array(features)

    # Convert strings of colors to integer values
    light_dict = {'red': 0, 'yellow': 1, 'green': 2}
    labels = np.array([light_dict[label] for label in labels])
    features, labels = shuffle(features, labels, random_state=42)

    return features, labels

In [ ]:
def create_base_cnn(input_shape=(32, 32, 3), num_classes=3):
    model = models.Sequential([

        # First convolutional layer
        layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), padding='same', data_format='channels_last', name='conv_1', activation='relu'
        ),
        # Second convolutional layer
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third convolutional layer
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten and Fully Connected Layers
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy']) # same as `tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')`

    return model
    

In [8]:
def train_model(X_train, y_train, X_test, y_test):

    # Data augmentation to prevent overfitting
    data_augmentation = models.Sequential([
        layers.RandomFlip('horizontal'),
        layers.RandomFlip('vertical'),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ])

    # Create base model without data autmentation
    model = create_base_cnn()

    model = models.Sequential([
        data_augmentation,
        model
    ])

    model.summary()

    early_stopping = callbacks.EarlyStopping(
        monitor='val_loss',
        restore_best_weights=True,
        patience=5
    )

    reduce_lr = callbacks.ReduceLROnPlatea(
        monitor='val_loss', 
        factor=0.2,
        patience=5, 
        min_lr=0.001
    )

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=32,
        callbacks=[early_stopping, redice_lr, WandbMetricsLogger()]
    )

    return model

In [ ]:
def main():
    
    # Example of generating random input data for testing
    import numpy as np
    x_test = np.random.rand(16, 64, 64, 3)  # 16 random images
    y_test = np.random.rand(16, 3)  # Random one-hot encoded labels
    y_test = y_test / y_test.sum(axis=1)[:, np.newaxis]  # Normalize to create valid probability distribution

    train_model()

if __name__ == "__main__":
    main()